<a href="https://colab.research.google.com/github/Think-JIn99/All-about-Market/blob/main/%EB%B6%84%EC%82%B0%ED%88%AC%EC%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import seaborn as sns  
import numpy as np
import sklearn
import sklearn.preprocessing
import tensorflow as tf
import requests
import matplotlib.pyplot as plt
%matplotlib inline 


In [163]:
#해당 링크는 한국거래소에서 상장법인목록을 엑셀로 다운로드하는 링크입니다.
#다운로드와 동시에 Pandas에 excel 파일이 load가 되는 구조입니다.
stock_code = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0] 
#stock_code.head()

stock_code.sort_values(['상장일'], ascending=True)

stock_code = stock_code[['회사명', '종목코드']] 

# 한글 컬럼명을 영어로 변경 
stock_code = stock_code.rename(columns={'회사명': 'company', '종목코드': 'code'}) 

# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
stock_code.code = stock_code.code.map('{:06d}'.format) 

In [164]:
class stock:
  def __init__(self, company):
    self.company = company
    self.df = pd.DataFrame()
  def make_df(self):
    code = stock_code[stock_code.company==self.company].code.values[0].strip() ## strip() : 공백제거
    a = 0
    #self.df = pd.DataFrame()
    for page in range(1,10+1):
        a += 1
        url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)     
        url = '{url}&page={page}'.format(url=url, page=page)
        if a%10 == 0:
          print(url)
        header = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 '}  
        res = requests.get(url,headers=header)
        self.df =self.df.append(pd.read_html(res.text, header=0)[0])
        
  def echo(self):
        print("class name:", self.__class__.__name__) #// 클래스이름을 출력한다

  def preprocessing(self):
    #self.df.dropna()를 이용해 결측값 있는 행 제거 
    self.df =self.df.dropna() 

      # 한글로 된 컬럼명을 영어로 바꿔줌 
    self.df =self.df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', 
                              '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'}) 
      # 데이터의 타입을 int형으로 바꿔줌 
    self.df[['close', 'diff', 'open', 'high', 'low', 'volume']] =self.df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int) 

      # 컬럼명 'date'의 타입을 date로 바꿔줌 
    self.df['date'] = pd.to_datetime(self.df['date']) 

      # 일자(date)를 기준으로 오름차순 정렬 
    self.df =self.df.sort_values(by=['date'], ascending=True) 

  def make_chart(self):
    plt.figure(figsize=(15,7))
    plt.plot(self.df['date'], self.df['close'])
    plt.title("STOCK")

  def breaking(self):
    temp = ((self.df['high'].iloc[:-1].values - self.df['low'].iloc[:-1].values) * 0.5 + self.df['open'].iloc[1:].values)
    temp = np.insert(temp, 0, np.nan)
    self.df['target'] = temp
    self.df['Percent'] = np.where(self.df['high'] > self.df['target'], self.df['close'] / self.df['target'], 1)
    self.df['Drop'] = (1 - self.df['Percent'] / self.df['Percent'].cummax()) * 100
    seed = 100000
    self.df['Profit'] = self.df['Percent'].cumprod() * seed

In [165]:
samsung = stock('삼성전자')
samsung.make_df()
samsung.preprocessing()
samsung.breaking()
samsung.df



http://finance.naver.com/item/sise_day.nhn?code=005930&page=10


,date,close,diff,open,high,low,volume,target,Percent,Drop,Profit
13,2021-10-06,71300,900,72600,72800,71200,18956962,NaN,1.000000,0.000000,100000.000000
12,2021-10-07,71600,300,71600,72100,71300,13683532,72400.0,1.000000,0.000000,100000.000000
11,2021-10-08,71500,100,72300,72400,71500,14043287,72700.0,1.000000,0.000000,100000.000000
10,2021-10-12,69000,2500,70700,70900,68700,31001484,71150.0,1.000000,0.000000,100000.000000
9,2021-10-13,68800,200,68700,69600,68300,24172015,69800.0,1.000000,0.000000,100000.000000
...,...,...,...,...,...,...,...,...,...,...,...
5,2022-02-23,73000,400,73800,73800,72800,10397964,74100.0,1.000000,1.268358,104748.815093
4,2022-02-24,71500,1500,72300,72300,71300,15759283,72800.0,1.000000,1.268358,104748.815093
3,2022-02-25,71900,400,72100,72600,71900,13168484,72600.0,1.000000,1.268358,104748.815093
2,2022-02-28,72100,200,71100,72100,71000,17781783,71450.0,1.009097,0.370169,105701.743432


In [166]:
SK = stock('SK하이닉스')
SK.make_df()
SK.preprocessing()
SK.breaking()


http://finance.naver.com/item/sise_day.nhn?code=000660&page=10


In [167]:
LG = stock('LG전자')
LG.make_df()
LG.preprocessing()
LG.breaking()


http://finance.naver.com/item/sise_day.nhn?code=066570&page=10


In [168]:
GS = stock('GS')
GS.make_df()
GS.preprocessing()
GS.breaking()


http://finance.naver.com/item/sise_day.nhn?code=078930&page=10


In [169]:
NAVER = stock('NAVER')
NAVER.make_df()
NAVER.preprocessing()
NAVER.breaking()


http://finance.naver.com/item/sise_day.nhn?code=035420&page=10


In [170]:
KB금융 = stock('KB금융')
KB금융.make_df()
KB금융.preprocessing()
KB금융.breaking()


http://finance.naver.com/item/sise_day.nhn?code=105560&page=10


In [171]:
셀트리온 = stock('셀트리온')
셀트리온.make_df()
셀트리온.preprocessing()
셀트리온.breaking()


http://finance.naver.com/item/sise_day.nhn?code=068270&page=10


In [172]:
SK바이오사이언스 = stock('SK바이오사이언스')
SK바이오사이언스.make_df()
SK바이오사이언스.preprocessing()
SK바이오사이언스.breaking()


http://finance.naver.com/item/sise_day.nhn?code=302440&page=10


In [173]:
카카오 = stock('카카오')
카카오.make_df()
카카오.preprocessing()
카카오.breaking()


http://finance.naver.com/item/sise_day.nhn?code=035720&page=10


In [174]:
넷마블 = stock('넷마블')
넷마블.make_df()
넷마블.preprocessing()
넷마블.breaking()


http://finance.naver.com/item/sise_day.nhn?code=251270&page=10


In [175]:
stocks = [samsung, SK, LG, GS, NAVER, KB금융, 셀트리온, SK바이오사이언스, 카카오, 넷마블]
for i in stocks:
  print(i.company)
  print(i.df.Profit)
  print()

삼성전자
13    100000.000000
12    100000.000000
11    100000.000000
10    100000.000000
9     100000.000000
          ...      
5     104748.815093
4     104748.815093
3     104748.815093
2     105701.743432
1     105701.743432
Name: Profit, Length: 100, dtype: float64

SK하이닉스
13    100000.000000
12    100000.000000
11    100000.000000
10    100000.000000
9     100054.377379
          ...      
5     106434.462842
4     106434.462842
3     106434.462842
2     106434.462842
1     105799.664852
Name: Profit, Length: 100, dtype: float64

LG전자
13    100000.000000
12     99598.393574
11     99598.393574
10     99397.994392
9      99397.994392
          ...      
5     104956.662192
4     102477.370960
3     102477.370960
2     102477.370960
1     101858.795280
Name: Profit, Length: 100, dtype: float64

GS
13    100000.000000
12    100000.000000
11    100000.000000
10    100278.551532
9     100278.551532
          ...      
5      99272.232561
4     100816.604567
3     100816.604567
2     10081

In [177]:
stocks = [samsung, SK, LG, GS, NAVER, KB금융, 셀트리온, SK바이오사이언스, 카카오, 넷마블]
a = 0
for i in stocks:
  a += i.df.Profit

print(a/10)


13    100000.000000
12    100369.979786
11    100574.991175
10    100759.940688
9     100752.289521
          ...      
5     101212.684709
4     100916.004361
3     101100.686085
2     101584.699593
1     101297.990866
Name: Profit, Length: 100, dtype: float64


# 잘못된 방법으로 했나?
거래량 가중 평균가는 후행성 지표이며, 이는 가격을 예측하는 특성이 없음을 의미합니다. 일부 트레이더는 하루 동안의 분석에 거래량 가중 평균가를 사용할 때 가장 유용하다고 주장합니다. 다른 시장 분석 도구와 마찬가지로, 거래량 가중 평균가는 단독으로 해석되어서는 안 되며, 다른 기술과 조합해 사용할 때 효과가 가장 좋습니다.